### Applying Lightweight Fine-Tuning to a Foundation Model

To pass this project, your code must:

 - Load a pre-trained model and evaluate its performance
 - Perform parameter-efficient fine-tuning using the pre-trained model
 - Perform inference using the fine-tuned model and compare its performance to the original model

Install libraries if not yet installed (Source: https://huggingface.co/docs/peft/main/en/task_guides/image_classification_lora)

In [1]:
!pip install transformers accelerate evaluate datasets peft -q
!pip install scikit-learn

You should consider upgrading via the 'c:\users\arnes\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\arnes\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


### Import

In [2]:
from transformers import AutoTokenizer
from peft import get_peft_model
from transformers import AutoModelForSequenceClassification
from peft import LoraConfig
from datasets import load_dataset
import evaluate

c:\Users\Arnes\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load fine-tuned model, tokenizer and dataset

In [3]:

# Load the dataset
dataset = load_dataset("dair-ai/emotion")
splits = ['train', 'validation', 'test']

# View dataset characteristics
print(dataset)

# print labels and their corresponding indices
print(dataset['train'].features['label'].str2int)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})
<bound method ClassLabel.str2int of ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)>


In [4]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained(
  "bert-base-uncased",
  num_labels=6,
  id2label={0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'},
  label2id={'sadness': 0, 'joy': 1, 'love': 3, 'anger': 4, 'fear': 5, 'surprise': 6}
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [6]:

config = LoraConfig()

# padding token
config.pad_token_id = 50256

peft_model = get_peft_model(model, config)

# how many trainable params
peft_model.print_trainable_parameters()



trainable params: 294,912 || all params: 109,781,766 || trainable%: 0.2686347749224584


###  Tokenize dataset

In [7]:
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], padding=True, truncation=True),
        batched=True
    )
# check first tokenized entry
print(tokenized_dataset["train"][0])

{'text': 'i didnt feel humiliated', 'label': 0, 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


### Define the metric for evaluation

In [8]:
# set up evaluation metric
metric = evaluate.load("accuracy")

# add metric to the trainer
def compute_metrics(eval_pred):
    # print for debugging purposes
    print("Compute Metric")
    print(eval_pred)
    predictions, labels = eval_pred
    predictions = predictions.argmax(-1)
    result = {"accuracy": metric.compute(predictions=predictions, references=labels)}
    print(result    )
    return result

### Load trainer from huggingface

In [9]:
# Load Huggingface Trainer class
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from accelerate import Accelerator

# Set up the Trainer
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

In [10]:
# print(tokenized_dataset["test"])

In [11]:
accelerator = Accelerator()
trainer.model = accelerator.prepare(trainer.model)

In [12]:

# get evaulation results
results = trainer.evaluate()
print(results)

"""
Above yields only this, accuracy is not printed
{'eval_runtime': 5.3402, 'eval_samples_per_second': 374.519, 'eval_steps_per_second': 23.407}

"""

100%|██████████| 125/125 [00:04<00:00, 25.05it/s]

{'eval_runtime': 5.6666, 'eval_samples_per_second': 352.944, 'eval_steps_per_second': 22.059}


"\nAbove yields only this, accuracy is not printed\n{'eval_runtime': 5.3402, 'eval_samples_per_second': 374.519, 'eval_steps_per_second': 23.407}\n\n"